In [ ]:
#| default_exp asink

In [ ]:
#| export
import json,asyncio
from litellm import acompletion, ModelResponse, stream_chunk_builder
from toolslm.funccall import call_func_async
from fastcore.utils import *
from lisette.core import *

In [ ]:
#| export
async def _lite_call_func_async(tc, ns, raise_on_err=True):
    res = await call_func_async(tc.function.name, json.loads(tc.function.arguments), ns=ns)
    return {"tool_call_id": tc.id, "role": "tool", "name": tc.function.name, "content": str(res)}

In [ ]:
#| export
class AsyncStreamResult:
    def __init__(self, agen, postproc=noop):
        self.agen = agen
        self.postproc = postproc
        self.complete = None
        
    async def __aiter__(self):
        chunks = []
        async for chunk in self.agen:
            chunks.append(chunk)
            yield chunk
        self.postproc(chunks)
        self.complete = stream_chunk_builder(chunks)

In [ ]:
#| export
class AsyncChat(Chat):
    async def _call(self, msg=None, stream=False, max_tool_rounds=1, tool_round=0, final_prompt=None, tool_choice=None, **kwargs):
        "Internal method that always yields responses"
        msgs = self._prepare_msgs(msg)
        res = await acompletion(model=self.model, messages=msgs, stream=stream,
                         tools=self.tool_schemas, temperature=self.temp, **kwargs)
        if stream:
            res = AsyncStreamResult(res, postproc=cite_footnotes)
            async for chunk in res: yield chunk
            res = res.complete
        
        yield res
        m = res.choices[0].message
        self.hist.append(m)

        if tcs := m.tool_calls:
            tool_results = [await _lite_call_func_async(tc, ns=self.ns) for tc in tcs]
            if tool_round>=max_tool_rounds-1:
                tool_results += ([{"role": "user", "content": final_prompt}] if final_prompt else [])
                tool_choice='none'
            async for result in self._call(
                tool_results, stream, max_tool_rounds, tool_round+1,
                final_prompt, tool_choice=tool_choice, **kwargs):
                    yield result
    
    async def __call__(self, msg=None, stream=False, max_tool_rounds=1, final_prompt=None, return_all=False, **kwargs):
        "Main call method - handles streaming vs non-streaming"
        if stream: return self._call(msg, stream, max_tool_rounds, 0, final_prompt, **kwargs)
        result_gen = self._call(msg, stream, max_tool_rounds, 0, final_prompt, **kwargs)
        if return_all: return [result async for result in result_gen] # toolloop behavior
        else: return [result async for result in result_gen][-1]      # normal chat behavior

## Async chat

In [ ]:
chat = AsyncChat(model="gpt-3.5-turbo")
await chat("What is 2+2?")

2+2 equals 4.

<details>

- id: `chatcmpl-CBijouwJTVhEI68vZKkpBiCEH3oYM`
- model: `gpt-3.5-turbo-0125`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=7, prompt_tokens=14, total_tokens=21, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None))`

</details>

## Async chat w tools

In [ ]:
async def async_add(a: int, b: int) -> int:
    "Add two numbers asynchronously"
    print('>>> async add is being called!')
    await asyncio.sleep(0.1)  # Simulate async work
    return a + b

In [ ]:
chat_with_tools = AsyncChat(model="claude-sonnet-4-20250514", tools=[async_add])
await chat_with_tools("What is 5 + 7? Use the tool to calculate it.")

async add is being called!


The result of 5 + 7 is 12.

<details>

- id: `chatcmpl-0a44c055-e3c2-4222-bcb3-b34c24cfb593`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=17, prompt_tokens=528, total_tokens=545, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0)`

</details>

## Streaming Async Chat

In [ ]:
chat = AsyncChat(model="claude-sonnet-4-20250514")
stream_gen = await chat("Count to 50", stream=True)

async for chunk in stream_gen:
    if isinstance(chunk, ModelResponse): display(chunk)
    else: print(delta_text(chunk) or '',end='')

## Streaming Async Chat w tools

In [ ]:
chat_stream_tools = AsyncChat(model="claude-sonnet-4-20250514", tools=[async_add])
stream_gen = await chat_stream_tools("What's 15 + 23? Use the tool and then explain the result.", stream=True)

async for chunk in stream_gen:
    if isinstance(chunk, ModelResponse): display(chunk)
    else: print(delta_text(chunk) or '', end='')